# Time Stamp Extract

Brief 1-2 sentence description of notebook.

TODO: Supplement the description
- Notebook that extracts the timestamps and gets the time that tones played

In [1]:
# Imports of all used packages and libraries
import sys
import os
import git
import glob
from collections import defaultdict

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/nancy/projects/reward_competition_extention'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
# Imports of all used packages and libraries
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
import spikeinterface.extractors as se
import spikeinterface.preprocessing as sp

In [7]:
import utilities.helper
import trodes.read_exported

## Inputs & Data

- Explanation of each input and where it comes from.

Inputs and Required data loading
- input variable names are in all caps snake case
- Whenever an input changes or is used for processing 
- The variables are all lower in snake case

In [8]:
# Path of the directory that contains the Spike Gadgets recording and the exported timestamp files
# Exported with this tool https://docs.spikegadgets.com/en/latest/basic/ExportFunctions.html
# Export these files:
    # -raw – Continuous raw band export.
    # -dio – Digital IO channel state change export.
    # -analogio – Continuous analog IO export.
INPUT_DIR = "/scratch/back_up/reward_competition_extention/data"
OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs
TONE_DIN = "dio_ECU_Din1"
TONE_STATE = 1
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_2"

In [9]:
# TODO: Find way not to hard code this
ALL_SESSION_DIR = glob.glob("/scratch/back_up/reward_competition_extention/data/omission/2023_06_18/*.rec")


In [10]:
ALL_SESSION_DIR

['/scratch/back_up/reward_competition_extention/data/omission/2023_06_18/20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1.rec']

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Other documentation

raw directory
- raw_group0.dat
    - voltage_value: Array with voltage measurement for each channel at each timestamp
- timestamps.dat
    - voltage_time_stamp: The time stamp of each voltage measurement

parent directory
- 1.videoTimeStamps.cameraHWSync
    - frame_number: Calculated by getting the index of each video time stamp tuple 
    - PosTimestamp: The time stamp of each video frame
    - HWframeCount: Unknown value. Starts at 30742 and increases by 1 for each tuple  
    - HWTimestamp: Unknown value. All zeroes
    - video_time: Calculated by dividing the frame number by the fps(frames per second) 
    - video_seconds: video_time, but rounded to seconds  	
    - These are filled in versions of the above collumns with the value from the most recent previous cell
        - filled_PosTimestamp 	
        - filledHWframeCount 	
        - filled_frame_number 	
        - filled_video_time 	
        - filled_video_seconds 	

DIO directory
- dio_ECU_Din1.dat
    - time: The time stamp the corresponds to the DIN input
    - state: Binary state of whether there is input from DIN or not 	
    - trial_number: Calculated by adding 1 to every time there is a DIN input
    - These are filled in versions of the above collumns with the value from the most recent previous cell
        - filled_state 	
        - filled_trial_number

ss_output directory (Spike sorting with Spike interface)
- firings.npz
    - unit_id: All the units that had a spike train for the given timestamp 	
    - number_of_units: Calculated by counting the number of units that had a spike train

## Functions

- function names are short and in snake case all lowercase
- a function name should be unique but does not have to describe the function
- doc strings describe functions not function names

In [11]:
def recursive_dictionary():
    return defaultdict(recursive_dictionary)

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [12]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# LOOP 1: Extracting all the Trodes

- Getting all the data from all the exported Trodes files

- Getting all the data from all the exported Trodes files and saving it to `session_to_trodes_data`
    - Creates a dictionary with the structure of:
        - `{dir_name: {file_name: metadata, file_name_2: metadata_2}, dir_name_2: {file_name_3: metadata_3, file_name_4: metadata_4}}`

In [13]:
# Saving the trodes data for each session
# Each key is a session name
# Each value is a dictionary of every recording file in that session
session_to_trodes_data = recursive_dictionary()


# Saving the path of the session recording
session_to_path = {}

# Going through each session recording
# Which includes all the recordings from all the miniloggers and cameras
for session_path in ALL_SESSION_DIR:   
    try:
        # Getting the name of the session from the path
        session_basename = os.path.splitext(os.path.basename(session_path))[0]
        print("Current Session: {}".format(session_basename))
        # Reading the trodes data for every recording file in the session directory
        session_to_trodes_data[session_basename] = trodes.read_exported.organize_all_trodes_export(session_path)
        
        session_to_path[session_basename] = session_path
    except Exception as e: 
        print(e)


Current Session: 20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1
Skipping file 20230618_100636_standard_comp_to_omission_D2_subj_1_4_t4b3L_box1_merged.timestampoffset.txt due to error: Settings format not supported


/nancy/projects/reward_competition_extention/src/trodes/read_exported.py:62: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  return np.dtype(dtype_spec)


Skipping file 20230618_100636_standard_comp_to_omission_D2_subj_1_1_t1b2L_box2_merged.timestampoffset.txt due to error: Settings format not supported


In [14]:
session_to_trodes_data

defaultdict(<function __main__.recursive_dictionary()>,
            {'20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1': defaultdict(dict,
                         {'20230618_100636_standard_comp_to_omission_D2_subj_1_1_t1b2L_box2_merged': {'raw': {'coordinates': {'description': 'Pad locations in microns',
                             'byte_order': 'little endian',
                             'original_file': '20230618_100636_standard_comp_to_omission_D2_subj_1_1_t1b2L_box2_merged.rec',
                             'clockrate': '20000',
                             'trodes_version': '2.4.0',
                             'compile_date': 'May 24 2023',
                             'compile_time': '10:11:04',
                             'qt_version': '6.2.2',
                             'commit_tag': 'heads/Release_2.4.0-0-g1eecf3b7',
                             'controller_firmware': '3.17',
                             'headstage_firmware': '2.2',
                      

- Adding the video timestamps

In [15]:
for session_path in ALL_SESSION_DIR:   
    try:
        session_basename = os.path.splitext(os.path.basename(session_path))[0]
        print("Current Session: {}".format(session_basename))
        file_to_video_timestamps = {}
        for video_timestamps in glob.glob(os.path.join(session_path, "*cameraHWSync")):
            video_basename = os.path.basename(video_timestamps)
            print("Current Video Name: {}".format(video_basename))
            timestamp_array = trodes.read_exported.read_trodes_extracted_data_file(video_timestamps)
            session_to_trodes_data[session_basename][session_basename]["video_timestamps"] = defaultdict(dict)
            session_to_trodes_data[session_basename][session_basename]["video_timestamps"][video_basename.split(".")[-3]] = timestamp_array
    
    
    except Exception as e: 
        print(e)

Current Session: 20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1
Current Video Name: 20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1.1.videoTimeStamps.cameraHWSync
Current Video Name: 20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1.2.videoTimeStamps.cameraHWSync


- Creating a dataframe the dictionary with a column for:
  - Session directory
  - Recording name
  - Metadata directory
  - Metadata file
  - And a column for each metadata

In [16]:
# Creating a dataframe from the nested dictionary
trodes_metadata_df = pd.DataFrame.from_dict({(i,j,k,l): session_to_trodes_data[i][j][k][l] 
                           for i in session_to_trodes_data.keys() 
                           for j in session_to_trodes_data[i].keys()
                           for k in session_to_trodes_data[i][j].keys()
                           for l in session_to_trodes_data[i][j][k].keys()},
                           orient='index')

# Resetting the index and renaming the columns
trodes_metadata_df = trodes_metadata_df.reset_index()
trodes_metadata_df = trodes_metadata_df.rename(columns={'level_0': 'session_dir', 'level_1': 'recording', 'level_2': 'metadata_dir', 'level_3': 'metadata_file'}, errors="ignore")

# Adding the session path to the dataframe
trodes_metadata_df["session_path"] = trodes_metadata_df["session_dir"].map(session_to_path)

In [17]:
trodes_metadata_df.head()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,timestamp_at_creation,first_timestamp,fields,data,direction,id,display_order,decimation,clock rate,session_path
0,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,raw,coordinates,Pad locations in microns,little endian,20230618_100636_standard_comp_to_omission_D2_s...,20000,2.4.0,May 24 2023,...,489210,835680,<ml int32><dv int32><ap int32>,"[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [...",NaN,NaN,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...
1,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,raw,timestamps,Raw timestamps,little endian,20230618_100636_standard_comp_to_omission_D2_s...,20000,2.4.0,May 24 2023,...,489210,835680,<time uint32>,"[[835680], [835681], [835682], [835683], [8356...",NaN,NaN,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...
2,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,DIO,dio_ECU_Dout3,State change data for one digital channel. Dis...,little endian,20230618_100636_standard_comp_to_omission_D2_s...,20000,2.4.0,May 24 2023,...,489210,835680,<time uint32><state uint8>,"[[835680, 0]]",output,ECU_Dout3,4,NaN,NaN,/scratch/back_up/reward_competition_extention/...
3,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230618_100636_standard_comp_to_omission_D2_s...,20000,2.4.0,May 24 2023,...,489210,835680,<time uint32><state uint8>,"[[835680, 0], [1934930, 1], [2134930, 0], [433...",input,ECU_Din1,7,NaN,NaN,/scratch/back_up/reward_competition_extention/...
4,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,DIO,dio_ECU_Dout1,State change data for one digital channel. Dis...,little endian,20230618_100636_standard_comp_to_omission_D2_s...,20000,2.4.0,May 24 2023,...,489210,835680,<time uint32><state uint8>,"[[835680, 0]]",output,ECU_Dout1,2,NaN,NaN,/scratch/back_up/reward_competition_extention/...


In [18]:
trodes_metadata_df.tail()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,timestamp_at_creation,first_timestamp,fields,data,direction,id,display_order,decimation,clock rate,session_path
20,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,DIO,dio_ECU_Din4,State change data for one digital channel. Dis...,little endian,20230618_100636_standard_comp_to_omission_D2_s...,20000,2.4.0,May 24 2023,...,489210,835680,<time uint32><state uint8>,"[[835680, 0]]",input,ECU_Din4,9,NaN,NaN,/scratch/back_up/reward_competition_extention/...
21,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,DIO,dio_ECU_Dout4,State change data for one digital channel. Dis...,little endian,20230618_100636_standard_comp_to_omission_D2_s...,20000,2.4.0,May 24 2023,...,489210,835680,<time uint32><state uint8>,"[[835680, 0]]",output,ECU_Dout4,5,NaN,NaN,/scratch/back_up/reward_competition_extention/...
22,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,DIO,dio_ECU_Dout1,State change data for one digital channel. Dis...,little endian,20230618_100636_standard_comp_to_omission_D2_s...,20000,2.4.0,May 24 2023,...,489210,835680,<time uint32><state uint8>,"[[835680, 0]]",output,ECU_Dout1,2,NaN,NaN,/scratch/back_up/reward_competition_extention/...
23,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230618_100636_standard_comp_to_omission_D2_s...,20000,2.4.0,May 24 2023,...,489210,835680,<time uint32><state uint8>,"[[835680, 0], [1934930, 1], [2134930, 0], [433...",input,ECU_Din1,7,NaN,NaN,/scratch/back_up/reward_competition_extention/...
24,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,video_timestamps,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,<PosTimestamp uint32><HWframeCount uint32><HWT...,"[[835678, 0, 0], [837064, 0, 0], [838450, 0, 0...",NaN,NaN,NaN,NaN,30000,/scratch/back_up/reward_competition_extention/...


- Getting the first item from each tuple in the arrays in the `data` column
  - This first item is usually just the timestamp

In [19]:
trodes_metadata_df["data"].iloc[0]

array([(0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0),
       (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0),
       (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0),
       (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0),
       (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0),
       (0, 0, 0), (0, 0, 0)],
      dtype=[('ml', '<i4'), ('dv', '<i4'), ('ap', '<i4')])

In [20]:
# Getting the dtype name of each column in the numpy array
trodes_metadata_df["first_dtype_name"] = trodes_metadata_df["data"].apply(lambda x: x.dtype.names[0])
# Getting the first item of each column in the numpy array
trodes_metadata_df["first_item_data"] = trodes_metadata_df["data"].apply(lambda x: x[x.dtype.names[0]])


In [21]:
# Same as above but for the last column
trodes_metadata_df["last_dtype_name"] = trodes_metadata_df["data"].apply(lambda x: x.dtype.names[-1])
trodes_metadata_df["last_item_data"] = trodes_metadata_df["data"].apply(lambda x: x[x.dtype.names[-1]])

In [22]:
trodes_metadata_df.head()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,direction,id,display_order,decimation,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data
0,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,raw,coordinates,Pad locations in microns,little endian,20230618_100636_standard_comp_to_omission_D2_s...,20000,2.4.0,May 24 2023,...,NaN,NaN,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...,ml,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",ap,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,raw,timestamps,Raw timestamps,little endian,20230618_100636_standard_comp_to_omission_D2_s...,20000,2.4.0,May 24 2023,...,NaN,NaN,NaN,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[835680, 835681, 835682, 835683, 835684, 83568...",time,"[835680, 835681, 835682, 835683, 835684, 83568..."
2,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,DIO,dio_ECU_Dout3,State change data for one digital channel. Dis...,little endian,20230618_100636_standard_comp_to_omission_D2_s...,20000,2.4.0,May 24 2023,...,output,ECU_Dout3,4,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,[835680],state,[0]
3,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230618_100636_standard_comp_to_omission_D2_s...,20000,2.4.0,May 24 2023,...,input,ECU_Din1,7,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[835680, 1934930, 2134930, 4334936, 4534932, 6...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ..."
4,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,DIO,dio_ECU_Dout1,State change data for one digital channel. Dis...,little endian,20230618_100636_standard_comp_to_omission_D2_s...,20000,2.4.0,May 24 2023,...,output,ECU_Dout1,2,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,[835680],state,[0]


In [23]:
trodes_metadata_df.tail()

,session_dir,recording,metadata_dir,metadata_file,description,byte_order,original_file,clockrate,trodes_version,compile_date,...,direction,id,display_order,decimation,clock rate,session_path,first_dtype_name,first_item_data,last_dtype_name,last_item_data
20,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,DIO,dio_ECU_Din4,State change data for one digital channel. Dis...,little endian,20230618_100636_standard_comp_to_omission_D2_s...,20000,2.4.0,May 24 2023,...,input,ECU_Din4,9,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,[835680],state,[0]
21,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,DIO,dio_ECU_Dout4,State change data for one digital channel. Dis...,little endian,20230618_100636_standard_comp_to_omission_D2_s...,20000,2.4.0,May 24 2023,...,output,ECU_Dout4,5,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,[835680],state,[0]
22,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,DIO,dio_ECU_Dout1,State change data for one digital channel. Dis...,little endian,20230618_100636_standard_comp_to_omission_D2_s...,20000,2.4.0,May 24 2023,...,output,ECU_Dout1,2,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,[835680],state,[0]
23,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,DIO,dio_ECU_Din1,State change data for one digital channel. Dis...,little endian,20230618_100636_standard_comp_to_omission_D2_s...,20000,2.4.0,May 24 2023,...,input,ECU_Din1,7,NaN,NaN,/scratch/back_up/reward_competition_extention/...,time,"[835680, 1934930, 2134930, 4334936, 4534932, 6...",state,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ..."
24,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,video_timestamps,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,30000,/scratch/back_up/reward_competition_extention/...,PosTimestamp,"[835678, 837064, 838450, 838921, 839835, 84122...",HWTimestamp,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [24]:
trodes_metadata_df["recording"].unique()

array(['20230618_100636_standard_comp_to_omission_D2_subj_1_1_t1b2L_box2_merged',
       '20230618_100636_standard_comp_to_omission_D2_subj_1_4_t4b3L_box1_merged',
       '20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1'],
      dtype=object)

In [25]:
trodes_metadata_df["all_subjects"] = trodes_metadata_df["recording"].apply(lambda x: x.replace("-","_").split("subj")[-1].split("t")[0].strip("_").replace("_",".").split(".and."))
trodes_metadata_df["all_subjects"] = trodes_metadata_df["all_subjects"].apply(lambda x: sorted([i.strip().strip(".") for i in x]))

In [26]:
trodes_metadata_df["current_subject"] = trodes_metadata_df["all_subjects"].apply(lambda x: x[0] if len(x) == 1 else np.nan)

In [27]:
trodes_metadata_df["all_subjects"]

0          [1.1]
1          [1.1]
2          [1.1]
3          [1.1]
4          [1.1]
5          [1.1]
6          [1.1]
7          [1.1]
8          [1.1]
9          [1.1]
10         [1.1]
11         [1.1]
12         [1.4]
13         [1.4]
14         [1.4]
15         [1.4]
16         [1.4]
17         [1.4]
18         [1.4]
19         [1.4]
20         [1.4]
21         [1.4]
22         [1.4]
23         [1.4]
24    [1.1, 1.4]
Name: all_subjects, dtype: object

In [28]:
trodes_metadata_df["current_subject"]

0     1.1
1     1.1
2     1.1
3     1.1
4     1.1
5     1.1
6     1.1
7     1.1
8     1.1
9     1.1
10    1.1
11    1.1
12    1.4
13    1.4
14    1.4
15    1.4
16    1.4
17    1.4
18    1.4
19    1.4
20    1.4
21    1.4
22    1.4
23    1.4
24    NaN
Name: current_subject, dtype: object

In [29]:
ALL_SUBJECTS_COL = "all_subjects"

In [30]:
trodes_metadata_df.to_pickle(os.path.join(OUTPUT_DIR, OUTPUT_PREFIX + "_trodes_metadata.pkl"))

In [31]:
trodes_metadata_df["metadata_dir"].unique()

array(['raw', 'DIO', 'analog', 'time', 'video_timestamps'], dtype=object)